In [ ]:
from src.stepper.serial_utilities.serial_utilities import (

    test_connection,
    print_ports_info,
    scan_ports,
    TestCase,
)

print_ports_info()
test_connection("COM3", 115200, test_case=TestCase.default())
scan_ports(test_case=TestCase.default())


In [ ]:
type(serial_connection)


In [9]:
serial_connection.close()


In [1]:
from src.stepper.device import Device

serial_connection = Device.SerialConnection("COM3", 115200, timeout=0.05)
device_params = Device.DeviceParams(serial_connection=serial_connection, address=1)
device = Device(device_params)

In [1]:
"""Test script for stepper device read and move functions."""

import time
from typing import Any

from stepper.device.device import Device
from stepper.stepper_core.parameters import DeviceParams, Direction, Speed


def test_read_functions(device: Device) -> dict[str, Any]:
    """Test all read-only functions of the device.
    
    :param device: Device instance to test
    :return: Dictionary containing all read values
    """
    results = {}
    
    # Version and system information
    results["version"] = device.version
    results["motor_rh"] = device.motor_rh
    results["bus_voltage"] = device.bus_voltage
    results["phase_current"] = device.phase_current
    
    # Position and movement related
    results["encoder_value"] = device.encoder_value
    results["pulse_count"] = device.pulse_count
    results["target_position"] = device.target_position
    results["open_loop_setpoint"] = device.open_loop_setpoint
    results["real_time_speed"] = device.real_time_speed
    results["real_time_position"] = device.real_time_position
    results["position_error"] = device.position_error
    
    # Status information
    results["status"] = device.status
    results["sys_status"] = device.sys_status
    results["is_enabled"] = device.is_enabled
    results["is_in_position"] = device.is_in_position
    results["is_stalled"] = device.is_stalled
    results["is_stall_protection_active"] = device.is_stall_protection_active
    
    # Homing status
    results["homing_status"] = device.homing_status
    results["encoder_ready"] = device.encoder_ready
    results["encoder_calibrated"] = device.encoder_calibrated
    results["is_homing"] = device.is_homing
    results["is_homing_failed"] = device.is_homing_failed
    
    return results


def test_basic_movement(device: Device) -> bool:
    """Test basic movement functions.
    
    :param device: Device instance to test
    :return: True if all tests pass
    """
    # Enable device
    if not device.enable():
        return False
    
    # Test jog functions
    print("Testing jog functions...")
    device.jog_at_speed(Speed(1000))
    device.wait(1)
    device.stop()
    
    device.jog_cw()
    device.wait(1)
    device.stop()
    
    device.jog_ccw()
    device.wait(1)
    device.stop()
    
    # Test timed jog
    print("Testing timed jog...")
    device.jog_time(2, Speed(1000))
    
    # Test relative moves
    print("Testing relative moves...")
    device.move_cw(1000)
    device.wait(2)
    device.move_ccw(1000)
    device.wait(2)
    
    # Test absolute move
    print("Testing absolute move...")
    device.move_to(0)
    device.wait(2)
    
    # Test homing
    print("Testing homing...")
    device.home()
    while device.is_homing:
        device.wait(0.5)
        print("Homing in progress...")
    
    # Disable device
    device.disable()
    
    return True


def test_debug_functions(device: Device) -> list[str]:
    """Test debug functions.
    
    :param device: Device instance to test
    :return: List of potential issues
    """
    return device.debug()


def main() -> None:
    """Main test function."""
    # Initialize device with default parameters

    serial_connection = Device.SerialConnection("COM3", 115200, timeout=0.05)
    device_params = Device.DeviceParams(serial_connection=serial_connection, address=1)
    device = Device(device_params)    
    
    print("Testing read functions...")
    read_results = test_read_functions(device)
    for key, value in read_results.items():
        print(f"{key}: {value}")
    
    print("\nTesting movement functions...")
    movement_success = test_basic_movement(device)
    print(f"Movement tests {'successful' if movement_success else 'failed'}")
    
    print("\nTesting debug functions...")
    issues = test_debug_functions(device)
    if issues:
        print("Detected issues:", issues)
    else:
        print("No issues detected")


if __name__ == "__main__":
    main()

Testing read functions...
version: VersionParams(firmware_version=2, hardware_version=130)
motor_rh: MotorRHParams(phase_resistance=2120, phase_inductance=3180, resistance_unit=<ResistanceUnit.mOhm: 1>, inductance_unit=<InductanceUnit.uH: 1>, _r_value=2120.0, _h_value=3180.0)
bus_voltage: BusVoltageParams(voltage=11516, unit=<VoltageUnit.mV: 1>, _value=11516.0)
phase_current: PhaseCurrentParams(current=38, unit=<CurrentUnit.mA: 1>, _value=38.0)
encoder_value: EncoderParams(encoder_value=60251, unit=<AngleUnit.deg: 182.04444444444445>, _value=330.9686279296875)
pulse_count: PulseCountParams(pulse_count=1850610, microsteps=16, _value=115663.125, _angle=208193.62499999997)
target_position: TargetPositionParams(position=37900486, unit=<AngleUnit.deg: 182.04444444444445>, _value=208193.58764648438)
open_loop_setpoint: OpenLoopTargetPositionParams(position=37900486, unit=<AngleUnit.deg: 182.04444444444445>, _value=208193.58764648438)
real_time_speed: RealTimeSpeedParams(speed=0, unit=<SpeedU

In [1]:
import time
from serial import Serial

from src.stepper.commands.get import (
    GetBusVoltage,
    GetConfig,
    GetEncoderValue,
    GetPositionError,
    GetMotorRH,
    GetPID,
    GetPhaseCurrent,
    GetRealTimePosition,
    GetRealTimeSpeed,
    GetStatus,
    GetSysStatus,
    GetTargetPosition,
    GetVersion,
)
from src.stepper.commands.home import (
    GetHomeStatus,
    Home,
    RetrieveHomeParam,
    SetHome,
    SetHomeParam,
    StopHome,
)
from src.stepper.commands.move import Disable, EStop, Enable, Jog, Move, SyncMove
from src.stepper.commands.set import (
    Kpid,
    LoopMode,
    Microstep,
    OpenLoopCurrent,
    SetConfig,
    SetID,
    SetLoopMode,
    SetMicrostep,
    SetOpenLoopCurrent,
    SetPID,
    SetReduction,
    SetStartSpeed,
    SpeedReduction,
    StartSpeedParams,
)
from src.stepper.commands.system import CalibrateEncoder, ZeroAllPositions
from src.stepper.stepper_core.configs import (
    AbsoluteFlag,
    Acceleration,
    Address,
    Direction,
    HomingMode,
    PulseCount,
    Speed,
    SyncFlag,
)
from src.stepper.stepper_core.parameters import DeviceParams, JogParams, PositionParams

serial_connection = Serial("COM3", 115200, timeout=0.1)

device_params = DeviceParams(
    serial_connection=serial_connection,
    address=Address(0x01),
)

position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(500),
    acceleration=Acceleration(127),
    pulse_count=PulseCount(160),
    absolute=AbsoluteFlag.RELATIVE,
)

jog_params = JogParams(
    direction=Direction.CW,
    speed=Speed(100),
    acceleration=Acceleration(255),
)

In [ ]:
SetHome(device=device_params).status

zero_all_positions = ZeroAllPositions
zero_all_positions.unlock()
zero_all_positions(device=device_params).status


In [28]:
device.move()


In [ ]:
position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(250),
    acceleration=Acceleration(200),
    pulse_count=PulseCount(160),
    absolute=AbsoluteFlag.RELATIVE,
)

Enable(device=device_params).status
Move(device=device_params, params=position_params).status
time.sleep(0.1)
print(GetPositionError(device=device_params).raw_data.data_dict)
print(GetRealTimePosition(device=device_params).raw_data.data_dict)
print(GetTargetPosition(device=device_params).raw_data.data_dict)


In [ ]:
Disable(device=device_params).status
Move(device=device_params, params=position_params).status
# This should show a CONDITIONAL_ERROR  

In [ ]:
Jog(device=device_params, params=jog_params).status

In [ ]:
EStop(device=device_params).status

In [ ]:
Move(device=device_params, params=position_params, setting=SyncFlag.SYNC).is_success
SyncMove(device=device_params).status

In [ ]:
position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(100),
    acceleration=Acceleration(255),
    pulse_count=PulseCount(6400),
    absolute=AbsoluteFlag.RELATIVE,
)

SetHome(device=device_params).status
Move(device=device_params, params=position_params).status



In [ ]:
Home(device=device_params, params=HomingMode.MULTI_TURN_UNLIMITED).status
# sleep(1)
# StopHome(device=device_params).status

In [ ]:
RetrieveHomeParam(device=device_params).data

In [13]:
homing_params = RetrieveHomeParam(device=device_params).raw_data


In [ ]:
homing_params.bytes

In [ ]:
SetHomeParam(device=device_params, params=homing_params).status

In [ ]:
Move(device=device_params, params=position_params).status



In [ ]:
Home(device=device_params, params=HomingMode.SINGLE_TURN_NEAREST).status

In [ ]:
GetHomeStatus(device=device_params).raw_data.data_dict

In [ ]:

calibrate_encoder = CalibrateEncoder
calibrate_encoder.unlock()
calibrate_encoder(device=device_params).status


In [ ]:

print(GetVersion(device=device_params).data)
print(GetMotorRH(device=device_params).data)
print(GetPID(device=device_params).data)
print(GetConfig(device=device_params).data)
print(GetStatus(device=device_params).data)
print(GetPositionError(device=device_params).data)
print(GetSysStatus(device=device_params).data)
print(GetBusVoltage(device=device_params).data)
print(GetPhaseCurrent(device=device_params).data)
print(GetEncoderValue(device=device_params).data)



In [ ]:
from itertools import chain

sys_status = GetSysStatus(device=device_params).raw_data
config_params = GetConfig(device=device_params).raw_data
for key, value in sorted(chain(sys_status.data_dict.items(), config_params.data_dict.items())):
    print(f"{key}: {value}")


In [ ]:
Set_Microstep = SetMicrostep
Set_Microstep.unlock()
Set_Microstep(device=device_params, params=Microstep(value=16)).status

In [ ]:
Set_ID = SetID
Set_ID.unlock()
Set_ID(device=device_params, params=Address(value=1)).status


In [ ]:
SetLoopMode = SetLoopMode
SetLoopMode.unlock()
SetLoopMode(device=device_params, params=LoopMode.CLOSED).status


In [ ]:
Set_open_loop_current = SetOpenLoopCurrent
Set_open_loop_current.unlock()
Set_open_loop_current(device=device_params, params=OpenLoopCurrent(value=800)).status

In [ ]:
PID_params = GetPID(device=device_params).raw_data
print(PID_params.data_dict)
Set_PID = SetPID
Set_PID.unlock()
Set_PID(device=device_params, params=PID_params).status


In [ ]:
from src.stepper.stepper_constants import EnablePin
start_speed_params = StartSpeedParams(
    direction=Direction.CW,
    speed=Speed(1000),
    acceleration=Acceleration(255),
    en_control=EnablePin.ENABLE,
)
Set_StartSpeed = SetStartSpeed
Set_StartSpeed.unlock()
Set_StartSpeed(device=device_params, params=start_speed_params).status

In [ ]:
from src.stepper.stepper_core.configs import StallTime

config_params = GetConfig(device=device_params).raw_data
print("Initial Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")

config_params.microstep = Microstep(value=32)
config_params.stall_time = StallTime(value=100)

Set_Config = SetConfig
Set_Config.unlock()
Set_Config(device=device_params, params=config_params).status

config_params = GetConfig(device=device_params).raw_data
print("-----------------------")
print("Changed Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")

config_params.microstep = Microstep(Microstep.default)
config_params.stall_time = StallTime(StallTime.default)

Set_Config = SetConfig
Set_Config.unlock()
Set_Config(device=device_params, params=config_params).status


config_params = GetConfig(device=device_params).raw_data
print("-----------------------")
print("Changed Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")

